In [1]:
!pip install mediapipe opencv-python


In [1]:
import ipywidgets as widgets
from IPython.display import display

camera_active = True

stop_button = widgets.Button(
    description='Stop Camera',
    button_style='danger',
    icon='stop'
)

def on_stop_button_clicked(b):
    global camera_active
    camera_active = False
    print("Camera stopped. Run the cell again to restart.")

stop_button.on_click(on_stop_button_clicked)
display(stop_button)


Button(button_style='danger', description='Stop Camera', icon='stop', style=ButtonStyle())

In [2]:
import cv2
import mediapipe as mp
import numpy as np
import time

mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

# Function to create rounded rectangle
def draw_rounded_rectangle(img, top_left, bottom_right, radius, color, thickness=-1):
    x1, y1 = top_left
    x2, y2 = bottom_right
    
    # Draw the filled rectangle without corners
    cv2.rectangle(img, (x1 + radius, y1), (x2 - radius, y1 + radius), color, thickness)  # Top
    cv2.rectangle(img, (x1, y1 + radius), (x2, y2 - radius), color, thickness)  # Middle
    cv2.rectangle(img, (x1 + radius, y2 - radius), (x2 - radius, y2), color, thickness)  # Bottom
    
    # Draw the four corner circles
    cv2.circle(img, (x1 + radius, y1 + radius), radius, color, thickness)  # Top-left
    cv2.circle(img, (x2 - radius, y1 + radius), radius, color, thickness)  # Top-right
    cv2.circle(img, (x1 + radius, y2 - radius), radius, color, thickness)  # Bottom-left
    cv2.circle(img, (x2 - radius, y2 - radius), radius, color, thickness)  # Bottom-right

cap = cv2.VideoCapture(0)

# Shoulder shrug counter variables
counter = 0 
stage = None
feedback = ""
left_shoulder_pos = 0
right_shoulder_pos = 0
baseline_left_y = 0
baseline_right_y = 0
baseline_set = False
calibration_frames = 0
calibration_sum_left = 0
calibration_sum_right = 0
shrug_threshold = 0.015  # Threshold for detecting shoulder shrug (percentage of height)
shrug_complete = False

# UI parameters
panel_width = 400
panel_height = 100
instruction_panel_height = 160  # Increased height for instructions
corner_radius = 20
bg_color = (30, 30, 30)  # Dark background for minimalist look
text_color = (255, 255, 255)  # White text
accent_color = (0, 200, 255)  # Cyan accent
progress_color = (0, 255, 200)  # Teal for progress

# Debug info
debug_mode = False  # Set to True to show position values

# Setup mediapipe instance
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            print("Failed to grab frame")
            break
            
        # Recolor image to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
      
        # Make detection
        results = pose.process(image)
    
        # Recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # Extract landmarks
        try:
            landmarks = results.pose_landmarks.landmark
            
            # Get coordinates for shoulder shrug detection
            left_shoulder = landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value]
            right_shoulder = landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value]
            left_ear = landmarks[mp_pose.PoseLandmark.LEFT_EAR.value]
            right_ear = landmarks[mp_pose.PoseLandmark.RIGHT_EAR.value]
            
            # Calculate shoulder positions (y-coordinate)
            left_shoulder_pos = left_shoulder.y
            right_shoulder_pos = right_shoulder.y
            
            # Frame height for normalization
            h, w, _ = image.shape
            
            # Set baseline shoulder position (calibration)
            if not baseline_set:
                # Collect samples for more stable baseline
                if calibration_frames < 30:  # Use 30 frames for calibration
                    calibration_sum_left += left_shoulder_pos
                    calibration_sum_right += right_shoulder_pos
                    calibration_frames += 1
                    feedback = f"Calibrating... {calibration_frames}/30"
                else:
                    baseline_left_y = calibration_sum_left / calibration_frames
                    baseline_right_y = calibration_sum_right / calibration_frames
                    baseline_set = True
                    feedback = "Ready. Shrug your shoulders!"
            else:
                # Calculate how much shoulders have moved (negative = up)
                left_delta = left_shoulder_pos - baseline_left_y
                right_delta = right_shoulder_pos - baseline_right_y
                
                # Average the movement of both shoulders
                avg_delta = (left_delta + right_delta) / 2
                
                # Debug info
                if debug_mode:
                    cv2.putText(image, f"L: {left_delta:.4f}", (10, 30), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1, cv2.LINE_AA)
                    cv2.putText(image, f"R: {right_delta:.4f}", (10, 60), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1, cv2.LINE_AA)
                    cv2.putText(image, f"Avg: {avg_delta:.4f}", (10, 90), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1, cv2.LINE_AA)
                    cv2.putText(image, f"Stage: {stage}", (10, 120), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1, cv2.LINE_AA)
                
                # SHOULDER SHRUG COUNTER LOGIC
                # Negative delta means shoulders moved up (since y-axis is inverted in image coordinates)
                
                # Shoulders up (shrug position)
                if avg_delta < -shrug_threshold and (stage == 'down' or stage is None):
                    stage = 'up'
                    feedback = "Shoulders up - good!"
                
                # Shoulders down (rest position)
                elif avg_delta > -0.005 and stage == 'up':  # Smaller threshold to ensure full return
                    stage = 'down'
                    counter += 1
                    feedback = "Rep counted!"
                
                # Initial state
                elif stage is None:
                    stage = 'down'
                    feedback = "Ready to start"
                
                # Current position feedback
                shoulder_gauge = -avg_delta * 100  # Convert to percentage for display
                
                # Clamp values for display
                shoulder_gauge = max(0, min(100, shoulder_gauge / (shrug_threshold * 3) * 100))
                
                if stage == 'up':
                    feedback = f"Hold shrug: {int(shoulder_gauge)}%"
                elif stage == 'down' and counter > 0:
                    feedback = "Return to rest position"
        
        except:
            pass
        
        # Get frame dimensions
        h, w, _ = image.shape
        
        # Create minimalist UI panels
        bottom_panel_x = (w - panel_width) // 2  # Center horizontally
        bottom_panel_y = h - panel_height - 20   # Bottom with margin
        
        top_panel_x = (w - panel_width) // 2     # Center horizontally
        top_panel_y = 20                         # Top with margin
        
        # Draw semi-transparent background for instruction panel (top)
        instruction_overlay = image.copy()
        draw_rounded_rectangle(
            instruction_overlay, 
            (top_panel_x, top_panel_y), 
            (top_panel_x + panel_width, top_panel_y + instruction_panel_height), 
            corner_radius, 
            bg_color, 
            -1
        )
        
        # Apply instruction overlay with transparency
        alpha = 0.8
        cv2.addWeighted(instruction_overlay, alpha, image, 1 - alpha, 0, image)
        
        # Add instruction title
        cv2.putText(image, "SHOULDER SHRUG EXERCISE", 
                   (top_panel_x + panel_width//2 - 130, top_panel_y + 30), 
                   cv2.FONT_HERSHEY_SIMPLEX, 0.7, accent_color, 1, cv2.LINE_AA)
        
        # Add step instructions - improved spacing and positioning
        instructions = [
            "1. Stand facing the camera in a relaxed position",
            "2. Lift both shoulders up toward your ears",
            "3. Hold briefly at the top position",
            "4. Lower shoulders back to resting position",
            "5. Repeat for desired number of repetitions"
        ]
        
        # More space between lines and smaller font
        line_spacing = 22
        start_y = top_panel_y + 60
        
        for i, instr in enumerate(instructions):
            y_pos = start_y + (i * line_spacing)
            cv2.putText(image, instr, 
                       (top_panel_x + 25, y_pos), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.45, text_color, 1, cv2.LINE_AA)
        
        # Draw semi-transparent background for counter panel (bottom)
        counter_overlay = image.copy()
        draw_rounded_rectangle(
            counter_overlay, 
            (bottom_panel_x, bottom_panel_y), 
            (bottom_panel_x + panel_width, bottom_panel_y + panel_height), 
            corner_radius, 
            bg_color, 
            -1
        )
        
        # Apply counter overlay with transparency
        alpha = 0.8
        cv2.addWeighted(counter_overlay, alpha, image, 1 - alpha, 0, image)
        
        # Draw shoulder height gauge - vertical bar
        gauge_width = 6
        gauge_x = bottom_panel_x + panel_width - 50
        gauge_height = panel_height - 40
        gauge_y = bottom_panel_y + 20
        
        # Background of gauge
        cv2.rectangle(image, 
                     (gauge_x, gauge_y), 
                     (gauge_x + gauge_width, gauge_y + gauge_height), 
                     (80, 80, 80), -1, cv2.LINE_AA)
        
        # Draw fill if calibrated
        if baseline_set:
            try:
                # Calculate fill height based on shoulder position
                shoulder_gauge = -((left_shoulder_pos + right_shoulder_pos) / 2 - 
                                   (baseline_left_y + baseline_right_y) / 2) * 100
                
                # Clamp values for display
                shoulder_gauge = max(0, min(100, shoulder_gauge / (shrug_threshold * 3) * 100))
                
                fill_height = int(gauge_height * (shoulder_gauge / 100))
                
                cv2.rectangle(image, 
                             (gauge_x, gauge_y + gauge_height - fill_height), 
                             (gauge_x + gauge_width, gauge_y + gauge_height), 
                             progress_color, -1, cv2.LINE_AA)
                
            except:
                pass
        
        # Add gauge labels
        cv2.putText(image, "UP", 
                   (gauge_x - 5, gauge_y - 5), 
                   cv2.FONT_HERSHEY_SIMPLEX, 0.4, text_color, 1, cv2.LINE_AA)
        
        cv2.putText(image, "DOWN", 
                   (gauge_x - 15, gauge_y + gauge_height + 15), 
                   cv2.FONT_HERSHEY_SIMPLEX, 0.4, text_color, 1, cv2.LINE_AA)
        
        # Add threshold marker on gauge
        threshold_y = gauge_y + gauge_height - int(gauge_height * ((shrug_threshold * 3 * 100) / 100) / 100)
        cv2.line(image, 
                (gauge_x - 5, threshold_y), 
                (gauge_x + gauge_width + 5, threshold_y), 
                (150, 150, 150), 1, cv2.LINE_AA)
        
        # Add "SHOULDER SHRUGS" label
        cv2.putText(image, "SHOULDER SHRUGS", 
                   (bottom_panel_x + panel_width//2 - 85, bottom_panel_y + 25), 
                   cv2.FONT_HERSHEY_SIMPLEX, 0.6, text_color, 1, cv2.LINE_AA)
        
        # Counter in smaller font and placed below the label
        cv2.putText(image, str(counter), 
                   (bottom_panel_x + panel_width//2 - (15 if counter < 10 else 25), bottom_panel_y + 55), 
                   cv2.FONT_HERSHEY_SIMPLEX, 1.2, text_color, 2, cv2.LINE_AA)
        
        # Current state and feedback
        if feedback:
            cv2.putText(image, feedback, 
                       (bottom_panel_x + 20, bottom_panel_y + 85), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.6, accent_color, 1, cv2.LINE_AA)
        
        # Draw key shoulder and ear landmarks for visualization with transparency
        if results.pose_landmarks:
            # Create overlay for landmarks
            landmarks_overlay = image.copy()
            
            # Key points to show
            key_landmarks = [
                mp_pose.PoseLandmark.LEFT_SHOULDER,
                mp_pose.PoseLandmark.RIGHT_SHOULDER,
                mp_pose.PoseLandmark.LEFT_EAR,
                mp_pose.PoseLandmark.RIGHT_EAR,
                mp_pose.PoseLandmark.LEFT_ELBOW,
                mp_pose.PoseLandmark.RIGHT_ELBOW
            ]
            
            # Draw key landmarks on overlay
            for landmark_id in key_landmarks:
                landmark = results.pose_landmarks.landmark[landmark_id.value]
                h, w, c = image.shape
                cx, cy = int(landmark.x * w), int(landmark.y * h)
                # Draw larger circles for key points
                cv2.circle(landmarks_overlay, (cx, cy), 5, accent_color, -1)
            
            # Draw connections between landmarks on overlay
            connections = [
                (mp_pose.PoseLandmark.LEFT_SHOULDER, mp_pose.PoseLandmark.RIGHT_SHOULDER),
                (mp_pose.PoseLandmark.LEFT_SHOULDER, mp_pose.PoseLandmark.LEFT_ELBOW),
                (mp_pose.PoseLandmark.RIGHT_SHOULDER, mp_pose.PoseLandmark.RIGHT_ELBOW),
                (mp_pose.PoseLandmark.LEFT_SHOULDER, mp_pose.PoseLandmark.LEFT_EAR),
                (mp_pose.PoseLandmark.RIGHT_SHOULDER, mp_pose.PoseLandmark.RIGHT_EAR),
            ]
            
            for connection in connections:
                start_point = results.pose_landmarks.landmark[connection[0].value]
                end_point = results.pose_landmarks.landmark[connection[1].value]
                
                start_x, start_y = int(start_point.x * w), int(start_point.y * h)
                end_x, end_y = int(end_point.x * w), int(end_point.y * h)
                
                cv2.line(landmarks_overlay, (start_x, start_y), (end_x, end_y), progress_color, 2)
            
            # Apply landmarks with transparency
            landmarks_alpha = 0.2  # 20% opacity (80% transparency)
            cv2.addWeighted(landmarks_overlay, landmarks_alpha, image, 1 - landmarks_alpha, 0, image)
        
        cv2.imshow('Shoulder Shrug Tracker', image)
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    
    cap.release()
    cv2.destroyAllWindows()